## PAIR PROGRAMMING LIMPIEZA V

### Valores Nulos Sklearn
---

In [176]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de **Limpieza III** donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I y II ya dejamos algunas columnas limpitas:


- age ✔️ la limpiamos en el pair de Limpieza II
- species ✔️ la limpiamos en el pair de Limpieza II
- country ✔️ la limpiamos en el pair de Limpieza II
- fatal ✔️ la limpiamos en el pair de Pandas V
- year ✔️ es una columna de tipo float deberíamos convertirla a integer
- sex ✔️ la limpiamos en el pair de Pandas V
- fecha ✔️ la limpiamos en el pair de Pandas V

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

In [177]:
df = pd.read_pickle("../files/attacks10.pkl")
df.head()

,case_number,year,mes,sexo,edades,country,type,activity,fatal,cat_species
0,1800.00.00,1997,NaN,F,NaN,seychelles,Unprovoked,a corsair's boat was overturned,Y,NaN
1,1797.05.28.R,1997,May,NaN,NaN,NaN,Unprovoked,Dropped overboard,Y,NaN
2,1792.09.12,1997,Sep,M,NaN,england,Provoked,Fishing,Y,NaN
3,1791.00.00,1997,NaN,F,NaN,australia,Unprovoked,NaN,Y,NaN
4,1788.05.10,1997,May,NaN,NaN,australia,Boat,Fishing,N,NaN


**EJERCICIO 1**

Es el momento de eliminar los nulos:

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [178]:
# Comprobamos el número de valores nulos de la columna 'edades'
df['edades'].isnull().sum() 

162

In [179]:
# Almacenamos en una variable todos los parámetros a aplicar por el método, en este caso cambiar los valores nulos por la media
imputer= SimpleImputer(strategy="mean",missing_values=np.nan) 

In [180]:
# Sobreescribimos y aplicamos el método .fit sobre la columna 'edades'
imputer= imputer.fit(df[['edades']]) 

In [181]:
# Sobreescribimos sobre la columna los cambios que aplicamos con el método .transform
df['edades'] = imputer.transform(df[['edades']]) 

In [182]:
# Comprobamos que no hay valores nulos
df['edades'].isnull().sum() 

0

---

- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
    💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [183]:
# Comprobamos el número de valores nulos de la columna 'sexo'
df['sexo'].isnull().sum() 

14

In [184]:
# Almacenamos en una variable todos los parámetros que queremos aplicar, en este caso cambiar los valores nulos por la moda
imputer= SimpleImputer(strategy="most_frequent",missing_values=np.nan)

In [185]:
# Sobreescribimos y aplicamos el método .fit sobre la columna 'sexo'
imputer= imputer.fit(df[['sexo']])

In [186]:
# Sobreescribimos sobre la columna los cambios que aplicamos con el método .transform
df['sexo'] = imputer.transform(df[['sexo']])

In [187]:
# Comprobamos que no hay valores nulos
df['sexo'].isnull().sum()

0

---

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [188]:
df['type'].isnull().sum() # Comprobamos los valores nulos de la columna 'type' y vemos que no hay ninguno

0

---

- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [189]:
# Guardamos en la variable numericas la selección de las columnas numéricas del DataFrame(DF) con el método .selec_dtypes
numericas= df.select_dtypes(include=np.number)
numericas.head()

,year,edades
0,1997,27.878808
1,1997,27.878808
2,1997,27.878808
3,1997,27.878808
4,1997,27.878808


In [190]:
# Al imputar el método KNN le damos 5 vecinos de comparación y aplicamos el método .fit a las columnas arriba seleccionadas
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(numericas)

KNNImputer()

In [191]:
# Aplicamos el siguiente paso con el método .transfrom
numericas_knn= imputerKNN.transform(numericas)

In [192]:
# Creamos un DF con las columnas numéricas actualizadas
df_knn_imputer = pd.DataFrame(numericas_knn, columns = numericas.columns)


In [193]:
# Sacamos el nombre de las columnas del DF sin nulos
columnas_knn = df_knn_imputer.columns

In [194]:
# Eliminamos esas columnas del DF
df.drop(columnas_knn, axis = 1, inplace = True)

In [195]:
# Creamos nuevas columnas en el DF original con en el DF creado con las numericas_trans
df[columnas_knn] = numericas_knn

In [196]:
# Comprobamos que no hay valores nulos en las nuevas columnas
df.isnull().sum()

case_number      0
mes            170
sexo             0
country         10
type             0
activity        31
fatal           97
cat_species    126
year             0
edades           0
dtype: int64

---

- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [197]:
# Abrimos de nuevo archivo inicial y lo guardamos en otro DataFrame para volver a reemplazar los valores nulos
df_2 = pd.read_pickle("../files/attacks10.pkl")
df_2.head()

,case_number,year,mes,sexo,edades,country,type,activity,fatal,cat_species
0,1800.00.00,1997,NaN,F,NaN,seychelles,Unprovoked,a corsair's boat was overturned,Y,NaN
1,1797.05.28.R,1997,May,NaN,NaN,NaN,Unprovoked,Dropped overboard,Y,NaN
2,1792.09.12,1997,Sep,M,NaN,england,Provoked,Fishing,Y,NaN
3,1791.00.00,1997,NaN,F,NaN,australia,Unprovoked,NaN,Y,NaN
4,1788.05.10,1997,May,NaN,NaN,australia,Boat,Fishing,N,NaN


In [198]:
# Guardamos en la variable numericas2 la selección de las columnas numéricas del DataFrame(DF) con el método .selec_dtypes
numericas2= df_2.select_dtypes(include=np.number)
numericas2.head()

,year,edades
0,1997,NaN
1,1997,NaN
2,1997,NaN
3,1997,NaN
4,1997,NaN


In [199]:
# Comprobamos los valores nulos de las columnas numéricas
numericas2.isnull().sum()

year        0
edades    162
dtype: int64

In [200]:
# Almacenamos en una variable todos los parámetros que queremos aplicar, en este caso cambiar los valores nulos por la media, redondeando a 3 decimales,
# iterando 20 veces de manera ascendente.
imputer2 = IterativeImputer(max_iter=20, n_nearest_features=None, imputation_order='ascending', initial_strategy= 'median', tol=3)

In [201]:
# Lo aplicamos con el método .fit
imputer2.fit(numericas2)

IterativeImputer(initial_strategy='median', max_iter=20, tol=3)

In [202]:
# Tranformamos los datos con el método .transform y convertimos el resultado el un DF
numericas_trans2 = pd.DataFrame(imputer2.transform(numericas2), columns = numericas2.columns)

In [203]:
# Sacamos el nombre de las columnas del DF sin nulos
columnas2 = numericas_trans2.columns

In [204]:
# Eliminamos esas columnas del DF
df_2.drop(columnas2, axis = 1, inplace = True)

In [205]:
# Creamos nuevas columnas en el DF original con en el DF creado con las numericas_trans2
df_2[columnas2] = numericas_trans2[columnas2]

In [206]:
# Comprobamos que no hay valores nulos en las nuevas columnas
df_2.isnull().sum()

case_number      0
mes            170
sexo            14
country         10
type             0
activity        31
fatal           97
cat_species    126
year             0
edades           0
dtype: int64

In [207]:
# Comprobamos los cambios finales en el DF 
df_2.head() 

,case_number,mes,sexo,country,type,activity,fatal,cat_species,year,edades
0,1800.00.00,NaN,F,seychelles,Unprovoked,a corsair's boat was overturned,Y,NaN,1997.0,28.421549
1,1797.05.28.R,May,NaN,NaN,Unprovoked,Dropped overboard,Y,NaN,1997.0,28.421549
2,1792.09.12,Sep,M,england,Provoked,Fishing,Y,NaN,1997.0,28.421549
3,1791.00.00,NaN,F,australia,Unprovoked,NaN,Y,NaN,1997.0,28.421549
4,1788.05.10,May,NaN,australia,Boat,Fishing,N,NaN,1997.0,28.421549


---

- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

    - Con el método de KNN Imputer no podemos asignarle parámetros para controlar el orden de comparación con las columnas, ni redondear decimales, ni especificar con que estadístico queremos reemplazar, cosa que con el método Iterative Imputer si podemos hacer.    
    - Nos da la impresión que según la cantidad de columnas numéricas(por ejemplo, si hay pocas) que el método KNN funciona con más precisión que el II.

---

In [208]:
nuevo_orden= ['case_number', 'year', 'mes','sexo','edades','country','type', 'activity','fatal','cat_species']

In [209]:
df2=df.reindex(columns = nuevo_orden) #Reordenamos las columnas.

In [210]:
df2.head()

,case_number,year,mes,sexo,edades,country,type,activity,fatal,cat_species
0,1800.00.00,1997.0,NaN,F,27.878808,seychelles,Unprovoked,a corsair's boat was overturned,Y,NaN
1,1797.05.28.R,1997.0,May,M,27.878808,NaN,Unprovoked,Dropped overboard,Y,NaN
2,1792.09.12,1997.0,Sep,M,27.878808,england,Provoked,Fishing,Y,NaN
3,1791.00.00,1997.0,NaN,F,27.878808,australia,Unprovoked,NaN,Y,NaN
4,1788.05.10,1997.0,May,M,27.878808,australia,Boat,Fishing,N,NaN


EJERCICIO 2

Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [211]:
df2.to_pickle('../files/attacks12_remplazo_nulos.pkl')